In [ ]:
import os
os.chdir("/opt/project")

import psutil
import numpy as np
from settings import (LOGS_PATH, MODELS_PATH, BACKUPS_PATH,
                      PREDICTIONS_PATH, TEST_DATA_PATH)

import keras
keras.mixed_precision.set_global_policy("mixed_float16")

from model import get_model
from performance import ModelPerformance
from datasets import ECGSequence, generate_label_files, get_files

In [ ]:
generate_label_files(num_classes=6, drop=1, verbose=True)

In [ ]:
# Model settings
val_split = 0.05
dataset_name = "tracings"
model_id = len(tuple(MODELS_PATH.iterdir())) - 1
worker_num = psutil.cpu_count(logical=True) - 2

# Optimization settings
lr = 0.001
batch_size = 64
opt = keras.optimizers.Adam(lr)
loss = keras.losses.BinaryCrossentropy()

callbacks = [
    # Learning Optimizers
    keras.callbacks.EarlyStopping(patience=9, min_delta=0.00001),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1,
                                      patience=7, min_lr=lr / 100),
    # Logs
    keras.callbacks.TensorBoard(log_dir=LOGS_PATH, write_graph=False),
    keras.callbacks.CSVLogger(LOGS_PATH / "training.log", append=False),
    # Checkpoints
    keras.callbacks.ModelCheckpoint(BACKUPS_PATH / f"model_last_{model_id}.keras"),
    keras.callbacks.ModelCheckpoint(BACKUPS_PATH / f"model_best_{model_id}.keras"),
]

train_seq, valid_seq = ECGSequence.get_train_and_val(
    get_files("input"), get_files("label"), dataset_name,
    batch_size, val_split, drop=1, shuffle=True,
    workers=worker_num, use_multiprocessing=True
)

# If you are continuing an interrupted section, uncomment line bellow:
# model = keras.models.load_model(MODELS_PATH / "model_{i}.keras", compile=False)
model = get_model(train_seq.n_classes)
model.compile(loss=loss, optimizer=opt)

In [ ]:
# Train neural network
# If you are continuing an interrupted section change initial epoch
history = model.fit(train_seq,
                    epochs=70,
                    initial_epoch=0,
                    callbacks=callbacks,
                    validation_data=valid_seq,
                    verbose=1)
# Save final result
model.save(MODELS_PATH / f"model_{model_id}.keras")

In [ ]:
# Predictions of the model on the test set
seq = ECGSequence([TEST_DATA_PATH / "ecg_tracings.hdf5"], None, dataset_name, batch_size=1)
y_score = model.predict(seq, verbose=1)

np.save(PREDICTIONS_PATH / f"prediction_{model_id}.npy", y_score)

In [ ]:
# Generate figures
stats = ModelPerformance(model_id)

stats.generate_table_two()
stats.generate_supplementary_figure_one()